In [259]:
# Import necessary libraries
import pandas as pd
import yfinance as yf
import datetime as dt
import numpy as np

In [ ]:
# Get S&P 500 tickers
sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_tickers = sp500_tickers.Symbol.to_list()
sp500_tickers

In [ ]:
# Clean Tickers of periods
sp500_tickers = [x.replace(".", "-") for x in sp500_tickers]
sp500_tickers

In [324]:
# Define Start and End dates for hist. data(5 yr)
end_date = dt.datetime.now().date().isoformat()
start_date = dt.datetime.now() - dt.timedelta(weeks=26)
start_date = start_date.date().isoformat()
start_date

'2021-02-11'

In [ ]:
sp_data = yf.download(sp500_tickers, period="5y", groupby="Ticker")
sp_data

In [ ]:
selection = yf.download(("GOOG", "AAPL", "WMT"), period="1mo")
selection

In [117]:
sp_data.to_csv("Sam/Untitled Folder")

In [ ]:
selection.loc[:, "Ticker"]

In [156]:
# def calcRSI (ticker):
#     data = yf.download(ticker, period="5y", groupby="Ticker")
#     data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
#     data["Price_Change"] = data["Adj Close"].pct_change()
#     data["Gains"] = data["Price_Change"].apply(lambda x: x if x > 0 else 0)
#     data["Losses"] = data["Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
#     data["Avg_Up"] = data["Gains"].ewm(span=19).mean()
#     data["Avg_Down"] = data["Losses"].ewm(span=19).mean()
#     data = data.dropna()
#     data["RS"] = data["Avg_Up"]/data["Avg_Down"]
#     data["RSI"] = data["RS"].apply(lambda x: 100 - (100/(x+1)))
#     return data

In [372]:
def calcRSI (ticker):
    data = yf.download(ticker, start=start_date,end=end_date, group_by="Ticker")
    data["Ticker"] = ticker
#     data["Moving_Avg_200_Days"] = data["Adj Close"].rolling(window=200).mean()
    data["Price_Change"] = data["Adj Close"].pct_change()
#     temp1 = data['Price_Change']
    data['Gains'] = data.loc[:,"Price_Change"].apply(lambda x: x if x > 0 else 0)
#     data["Gains"] = data.iloc[:,7].apply(lambda x: x if x > 0 else 0)
    data["Losses"] = data.loc[:,"Price_Change"].apply(lambda x: abs(x) if x < 0 else 0)
#     data["Losses"] = data.iloc[:,7].apply(lambda x: abs(x) if x < 0 else 0)
#     temp2 = data["Gains"]
#     temp3 = data["Losses"]
    data["Avg_Up"] = data.loc[:,"Gains"].ewm(com=13,adjust=False, min_periods=14).mean()
#     temp4 = data["Avg_Up"]
    data["Avg_Down"] = data.loc[:, "Losses"].ewm(com=13,adjust=False, min_periods=14).mean()
#     temp5 = data["Avg_Down"]

#     temp6 = temp4/temp5
    data["RS"] = data.loc[:, "Avg_Up"]/data.loc[:,"Avg_Down"]
    data["RSI"] = data.loc[:,"RS"].apply(lambda x: 100 - (100/(x+1)))
    data = data.dropna()
#     temp7 = data["RSI"]
#     output_df = pd.DataFrame(columns=["Ticker", "RSI"])
#     output_df = output_df.append({"Ticker" : ticker,
#                                  "RSI" : temp7},
#                                 ignore_index=True)
    today = data.iloc[-1]
#     output_df = pd.DataFrame(columns=["Ticker","RSI"])
#     output_df = pd.merge(output_df,today[["Ticker", "RSI"]], on="Ticker", how="left")
    return today

In [373]:
calcRSI(sp500_tickers[0])

[*********************100%***********************]  1 of 1 completed


Open                 200.0
High            201.770004
Low             199.309998
Close           201.570007
Adj Close       201.570007
Volume             2217400
Ticker                 MMM
Price_Change      0.011644
Gains             0.011644
Losses                 0.0
Avg_Up            0.004301
Avg_Down          0.003314
RS                 1.29774
RSI              56.478966
Name: 2021-08-11 00:00:00, dtype: object

NameError: name 'output_df' is not defined

In [160]:
sp5_tickers= ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']

In [200]:
join_data = pd.DataFrame(index="Tickers",columns="RSI")

In [444]:
def Todays_RSI_Hi():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp500_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_hi = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_hi.sort_values(by=["RSI"], ascending=False,inplace=True)

    return rsi_df_hi

In [445]:
fig = Todays_RSI_Hi()

[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
2
[*********************100%***********************]  1 of 1 completed
3
[*********************100%***********************]  1 of 1 completed
4
[*********************100%***********************]  1 of 1 completed
5
[*********************100%***********************]  1 of 1 completed
6
[*********************100%***********************]  1 of 1 completed
7
[*********************100%***********************]  1 of 1 completed
8
[*********************100%***********************]  1 of 1 completed
9
[*********************100%***********************]  1 of 1 completed
10
[*********************100%***********************]  1 of 1 completed
11
[*********************100%***********************]  1 of 1 completed
12
[*********************100%***********************]  1 of 1 completed
13
[*********************100%***********************]  1 of 1 completed
14
[

In [446]:
fig

,RSI
DHR,83.861249
MSCI,82.379362
DGX,81.627377
DG,81.374252
GRMN,81.259517
...,...
HUM,27.770298
CNC,25.762915
BAX,25.760722
INCY,25.594658


In [440]:
def Todays_RSI_Low():
    stock_tick_list = []
    stock_rsi_list = []
    count = 0
#     join_data = pd.DataFrame()
    for stock in sp5_tickers:
        stockRSI = calcRSI(stock)
        stock_tick_list.append(stockRSI["Ticker"])
        stock_rsi_list.append(stockRSI["RSI"])
        count+=1
        print(count)
    rsi_df_low = pd.DataFrame(data=(stock_rsi_list), index=stock_tick_list, columns=["RSI"])
    rsi_df_low = rsi_df_low.sort_values(by=["RSI"],ascending=True)

    return rsi_df_low

In [442]:
fig2 = Todays_RSI_Low()

[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
2
[*********************100%***********************]  1 of 1 completed
3
[*********************100%***********************]  1 of 1 completed
4
[*********************100%***********************]  1 of 1 completed
5


In [443]:
fig2

,RSI
ABBV,41.477121
ABMD,46.414618
MMM,56.478966
ABT,58.895960
ACN,63.691639


In [ ]:
aapl_data = yf.download('AAPL', period="1mo")
aapl_data

In [140]:
aapl_data['Ticker'] = 'AAPL'
aapl_data

,Open,High,Low,Close,Adj Close,Volume,Ticker
Date,,,,,,,
2021-07-12,146.210007,146.320007,144.000000,144.500000,144.283829,76299700,AAPL
2021-07-13,144.029999,147.460007,143.630005,145.639999,145.422119,100827100,AAPL
2021-07-14,148.100006,149.570007,147.679993,149.149994,148.926865,127050800,AAPL
2021-07-15,149.240005,150.000000,147.089996,148.479996,148.257874,106820300,AAPL
2021-07-16,148.460007,149.759995,145.880005,146.389999,146.170990,93251400,AAPL
2021-07-19,143.750000,144.070007,141.669998,142.449997,142.236893,121434600,AAPL
2021-07-20,143.460007,147.100006,142.960007,146.149994,145.931351,96350000,AAPL
2021-07-21,145.529999,146.130005,144.630005,145.399994,145.182480,74993500,AAPL
2021-07-22,145.940002,148.199997,145.809998,146.800003,146.580383,77338200,AAPL


In [127]:
wmt_data = yf.download("WMT", period="1mo")
wmt_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-12,140.690002,140.789993,139.330002,140.050003,140.050003,6505000
2021-07-13,139.990005,141.179993,139.940002,140.580002,140.580002,6315900
2021-07-14,140.710007,141.800003,139.830002,141.550003,141.550003,6270700
2021-07-15,142.000000,142.639999,141.089996,141.660004,141.660004,5991600
2021-07-16,141.699997,142.130005,140.949997,141.559998,141.559998,6031800
2021-07-19,141.410004,142.979996,139.770004,141.229996,141.229996,9127700
2021-07-20,140.970001,142.470001,140.779999,141.869995,141.869995,6340100
2021-07-21,142.500000,142.539993,140.710007,141.169998,141.169998,6197300
2021-07-22,141.130005,142.119995,140.800003,141.270004,141.270004,4337800


In [137]:
combined = pd.merge(aapl_data, wmt_data, how="left", on=['Date'])

In [138]:
combined

,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y
Date,,,,,,,,,,,,
2021-07-12,146.210007,146.320007,144.000000,144.500000,144.283829,76299700,140.690002,140.789993,139.330002,140.050003,140.050003,6505000
2021-07-13,144.029999,147.460007,143.630005,145.639999,145.422119,100827100,139.990005,141.179993,139.940002,140.580002,140.580002,6315900
2021-07-14,148.100006,149.570007,147.679993,149.149994,148.926865,127050800,140.710007,141.800003,139.830002,141.550003,141.550003,6270700
2021-07-15,149.240005,150.000000,147.089996,148.479996,148.257874,106820300,142.000000,142.639999,141.089996,141.660004,141.660004,5991600
2021-07-16,148.460007,149.759995,145.880005,146.389999,146.170990,93251400,141.699997,142.130005,140.949997,141.559998,141.559998,6031800
2021-07-19,143.750000,144.070007,141.669998,142.449997,142.236893,121434600,141.410004,142.979996,139.770004,141.229996,141.229996,9127700
2021-07-20,143.460007,147.100006,142.960007,146.149994,145.931351,96350000,140.970001,142.470001,140.779999,141.869995,141.869995,6340100
2021-07-21,145.529999,146.130005,144.630005,145.399994,145.182480,74993500,142.500000,142.539993,140.710007,141.169998,141.169998,6197300
2021-07-22,145.940002,148.199997,145.809998,146.800003,146.580383,77338200,141.130005,142.119995,140.800003,141.270004,141.270004,4337800


In [ ]:
""